/home/lgrose/miniconda3/envs/loop/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)


array([[ 1,  2,  3,  4],
       [ 0, -1, -1, -1],
       [ 0, -1, -1, -1],
       [ 0, -1, -1, -1],
       [ 0, -1, -1, -1]])

array([[1, 1, 1, 3],
       [1, 2, 3, 1],
       [0, 1, 1, 1],
       [1, 3, 2, 1],
       [1, 3, 1, 2]])